In [15]:
!gdown --id 1RceyQijuEoLvbsRILQZEQ1X0Unsi45pN

Downloading...
From: https://drive.google.com/uc?id=1RceyQijuEoLvbsRILQZEQ1X0Unsi45pN
To: /content/ham10000_dataset.zip
711MB [00:06, 117MB/s]


In [51]:
!gdown --id 1-0WKOSw96PZ2Ww9mhLhIGBehePZnQciQ

Downloading...
From: https://drive.google.com/uc?id=1-0WKOSw96PZ2Ww9mhLhIGBehePZnQciQ
To: /content/model_main
181MB [00:01, 168MB/s]


In [16]:
!unzip *.zip

Streaming output truncated to the last 5000 lines.
  inflating: ds/train/nv/ISIC_0033148.jpg.jpg  
  inflating: ds/train/nv/ISIC_0032434.jpg.jpg  
  inflating: ds/train/nv/ISIC_0029628.jpg.jpg  
  inflating: ds/train/nv/ISIC_0026359.jpg.jpg  
  inflating: ds/train/nv/ISIC_0033188.jpg.jpg  
  inflating: ds/train/nv/ISIC_0032354.jpg.jpg  
  inflating: ds/train/nv/ISIC_0028292.jpg.jpg  
  inflating: ds/train/nv/ISIC_0026752.jpg.jpg  
  inflating: ds/train/nv/ISIC_0027520.jpg.jpg  
  inflating: ds/train/nv/ISIC_0032332.jpg.jpg  
  inflating: ds/train/nv/ISIC_0028668.jpg.jpg  
  inflating: ds/train/nv/ISIC_0029888.jpg.jpg  
  inflating: ds/train/nv/ISIC_0032729.jpg.jpg  
  inflating: ds/train/nv/ISIC_0028157.jpg.jpg  
  inflating: ds/train/nv/ISIC_0024327.jpg.jpg  
  inflating: ds/train/nv/ISIC_0028599.jpg.jpg  
  inflating: ds/train/nv/ISIC_0030900.jpg.jpg  
  inflating: ds/train/nv/ISIC_0030355.jpg.jpg  
  inflating: ds/train/nv/ISIC_0027944.jpg.jpg  
  inflating: ds/train/nv/ISIC_0024721

In [3]:
!pip install flask-ngrok
!pip install flask==0.12.2

     |████████████████████████████████| 92kB 4.1MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [18]:
from tensorflow import keras
import os
from keras.models import Model
import cv2
import yaml
import numpy as np
import flask
from flask import Flask,render_template, make_response, flash
from flask_ngrok import run_with_ngrok
from keras.applications.vgg19 import preprocess_input
import keras.backend as K

In [8]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [9]:
loss_fun = weighted_categorical_crossentropy(np.array([0.13472377, 0.08570948, 0.04008614, 0.3830841 , 0.03958192,0.00657042, 0.31024417]))

In [14]:
model_main = keras.models.load_model('/content/model_main', custom_objects={'loss':loss_fun})

In [45]:
dictionary = {
    0: 'Actinic keratoses and intraepithelial carcinoma',
    1: 'basal cell carcinoma',
    2: 'benign keratosis-like lesions',
    4: 'melanoma',
    3: 'dermatofibroma',
    5: 'melanocytic nevi',
    6: 'vascular lesions'
}

In [49]:
def classify_image(img):
    img = cv2.resize(img,(256,256))
    img = preprocess_input(img)
    img = img[np.newaxis,:,:,:]
    val = model.predict(img).argmax(axis=-1)
    return dictionary.get(val[0])

In [50]:
data = cv2.imread('/content/ds/train/akiec/ISIC_0024329.jpg.jpg')
classify_image(data)

'benign keratosis-like lesions'